In [1]:
# Testing how TF Checkpoints work

import tensorflow as tf
from tensorflow.keras import layers, Sequential
from shape_completion_training.my_keras_model import MyKerasModel
from link_bot_pycommon.get_scenario import get_scenario
from moonshine.gpu_config import limit_gpu_mem
import rospy
import numpy as np
np.set_printoptions(precision=3, suppress=True)
rospy.init_node("testing")
limit_gpu_mem(2.0)

In [2]:
from my_cfm.cfm import CFM

In [3]:
x = tf.ones([6,1])

In [4]:
# class _CFM(MyKerasModel):

#     def __init__(self):
#         super().__init__({}, 1)
#         self.encoder = _Encoder()
#         self.dynamics = _LocallyLinearPredictor()
#         self.observer = _Observer()

#     def call(self, x):
#         # forward pass
#         z, z_pos = self.encoder(x), self.encoder(x)  # b x z_dim
#         z_seq = self.dynamics(z)
#         y_seq = self.observer(z_pos)
#         return z_seq, y_seq

# class _Encoder(MyKerasModel):

#     def __init__(self):
#         super().__init__({}, 1)
#         self.encoder_sequtial = Sequential([
#             layers.Conv2D(filters=64, kernel_size=3),
#             layers.LeakyReLU(0.2),
#             layers.Conv2D(filters=64, kernel_size=4, strides=2),
#             layers.LeakyReLU(0.2),
#             # 64 x 32 x 32
#             layers.Conv2D(filters=64, kernel_size=3, strides=1),
#             layers.LeakyReLU(0.2),
#             layers.Conv2D(filters=128, kernel_size=4, strides=2),
#             layers.LeakyReLU(0.2),
#             # 128 x 16 x 16
#             layers.Conv2D(filters=256, kernel_size=4, strides=2),
#             layers.LeakyReLU(0.2),
#             # Option 1: 256 x 8 x 8
#             layers.Conv2D(filters=256, kernel_size=4, strides=2),
#             layers.LeakyReLU(0.2),
#             # 256 x 4 x 4
#         ], name='encoder_sequential')
#         self.z_dim = 32
#         self.out = layers.Dense(self.z_dim)

#     # @tf.function
#     def call(self, x):
#         h = self.encoder_sequtial(x)
#         z = self.out(h)
#         return z

# class _LocallyLinearPredictor(MyKerasModel):

#     def __init__(self):
#         super().__init__({}, 1)
#         self.z_dim = 32

#         my_layers = []
#         for h in [32,32]:
#             my_layers.append(layers.Dense(h, activation="relu"))
#         my_layers.append(layers.Dense(self.z_dim * self.z_dim, activation=None))

#         self.dynamics_sequential = Sequential(my_layers, name='predictor_sequential')

#     # @tf.function
#     def call(self, x):
#         z = self.dynamics_sequential(x)
#         return z


# class _Observer(MyKerasModel):

#     def __init__(self):
#         super().__init__({}, 1)
#         my_layers = []
#         for h in [32,32]:
#             my_layers.append(layers.Dense(h, activation="relu"))
#         final_dim = 16
#         my_layers.append(layers.Dense(final_dim, activation=None))

#         self.observer_sequential = Sequential(my_layers, name='observer_sequential')

#     # @tf.function
#     def call(self, x):
#         y = self.observer_sequential(x)
#         return y


In [5]:
# class MyModel(tf.keras.layers.Layer):
    
#     def __init__(self):
#         super().__init__(self)
#         self.encoder = Encoder()
        
#     def call(self, x):
#         return self.encoder(x)
        
# class Encoder(tf.keras.layers.Layer):
    
#     def __init__(self):
#         super().__init__(self)
#         self.s = tf.keras.Sequential([tf.keras.layers.Dense(10), tf.keras.layers.Dense(10)])
        
#     def call(self, x):
#         return self.s(x)

In [6]:
# model = MyModel()
# model = _CFM()
scenario = get_scenario("dual_floating")
hparams = {
  "model_class": "CFM",
  "sequence_length": 10,
  "state_keys": [
    "z"
  ],
  "observation_feature_keys": [
    "rope"
  ],
  "obs_keys": [
    "color_depth_image"
  ],
  "action_keys": [
    "left_gripper_position",
    "right_gripper_position"
  ],
  "fc_layer_sizes": [
    64,
    64
  ],
  "z_dim": 32,
  "learning_rate": 0.001,
  "epochs": 1,
  "use_observation_feature_loss": False,
  "dynamics_dataset_hparams": {
    "nickname": "cfm",
    "robot_namespace": "none",
    "seed": 34,
    "n_trajs": 4096,
    "data_collection_params": {
      "extent": [
        0.45,
        1.2,
        -0.6,
        0.6,
        0.6,
        1.3
      ],
      "left_gripper_action_sample_extent": [
        0.45,
        1.2,
        -0.6,
        0.6,
        0.6,
        1.3
      ],
      "right_gripper_action_sample_extent": [
        0.45,
        1.2,
        -0.6,
        0.6,
        0.6,
        1.3
      ],
      "res": 0.02,
      "max_step_size": 0.01,
      "steps_per_traj": 2,
      "no_objects": True,
      "randomize_n": None,
      "repeat_delta_gripper_motion_probability": 0.8,
      "max_distance_gripper_can_move": 0.1,
      "max_distance_between_grippers": 0.55
    },
    "states_description": {},
    "action_description": {
      "left_gripper_position": 3,
      "right_gripper_position": 3
    },
    "observations_description": {
      "left_gripper": 3,
      "right_gripper": 3,
      "color_depth_image": [
        126,
        224,
        4
      ]
    },
    "observation_features_description": {
      "rope": 75
    },
    "scenario": "dual_floating",
    "scenario_metadata": {}
  },
  "batch_size": 128,
  "seed": 0,
  "datasets": [
    "fwd_model_data/cfm_1602196483_34c0a63f25_4096+L2+nonan"
  ],
  "latest_training_time": 1602298809
    }
model = CFM(hparams, 1, scenario)
example = {
    'color_depth_image': tf.zeros([1,2,90,160,4]),
    'batch_size': tf.constant(1),
    'left_gripper_position': tf.zeros([1,1,3]),
    'right_gripper_position': tf.zeros([1,1,3]),
}
_ = model(example)
wts = model.get_weights()
wts[0] = wts[0] * 0 + 2
model.set_weights(wts)

In [7]:
ckpt = tf.train.Checkpoint(kwd=model)
ckpt.save("my-ckpt")

'my-ckpt-1'

In [8]:
w = model.get_weights()
print(w[0][0,0,0])

[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


In [9]:
# model2 = MyModel()
# model2 = _CFM()
model2 = CFM(hparams, 1, scenario)

In [10]:
# class Not(tf.keras.Model):
#     def __init__(self, a):
#         super().__init__(self)
#         self.actually = a
# not_model2  = Not(model2)

In [11]:
ckpt2 = tf.train.Checkpoint(kwd=model2)
status = ckpt2.restore("my-ckpt-1")
_ = model2(example)

In [12]:
w = model2.get_weights()
print(w[0][0,0,0])

[2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


In [13]:
status.assert_existing_objects_matched()
status.assert_consumed()

In [14]:
print(dir(ckpt2))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_variable_with_custom_getter', '_attached_dependencies', '_checkpoint_dependencies', '_deferred_dependencies', '_gather_saveables_for_checkpoint', '_handle_deferred_dependencies', '_list_extra_dependencies_for_serialization', '_list_functions_for_serialization', '_lookup_dependency', '_map_resources', '_maybe_create_save_counter', '_maybe_initialize_trackable', '_name_based_attribute_restore', '_name_based_restores', '_no_dependency', '_object_identifier', '_preload_simple_restoration', '_restore_from_checkpoint_position', '_save_assign_op', '_save_counter', '_saver', '_self_name_based_restores', '_self_saveable_object_factories', '_self_seta

In [15]:
tf.train.list_variables('my-ckpt-1')

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('kwd/dynamics/dynamics_sequential/layer_with_weights-0/bias/.ATTRIBUTES/VARIABLE_VALUE',
  [64]),
 ('kwd/dynamics/dynamics_sequential/layer_with_weights-0/kernel/.ATTRIBUTES/VARIABLE_VALUE',
  [38, 64]),
 ('kwd/dynamics/dynamics_sequential/layer_with_weights-1/bias/.ATTRIBUTES/VARIABLE_VALUE',
  [64]),
 ('kwd/dynamics/dynamics_sequential/layer_with_weights-1/kernel/.ATTRIBUTES/VARIABLE_VALUE',
  [64, 64]),
 ('kwd/dynamics/dynamics_sequential/layer_with_weights-2/bias/.ATTRIBUTES/VARIABLE_VALUE',
  [1024]),
 ('kwd/dynamics/dynamics_sequential/layer_with_weights-2/kernel/.ATTRIBUTES/VARIABLE_VALUE',
  [64, 1024]),
 ('kwd/encoder/encoder_sequtial/layer_with_weights-0/bias/.ATTRIBUTES/VARIABLE_VALUE',
  [64]),
 ('kwd/encoder/encoder_sequtial/layer_with_weights-0/kernel/.ATTRIBUTES/VARIABLE_VALUE',
  [3, 3, 4, 64]),
 ('kwd/encoder/encoder_sequtial/layer_with_weights-1/bias/.ATTRIBUTES/VARIABLE_VALUE',
  [64]),
 ('kwd/encoder/encoder_sequtial/layer_wi

In [16]:
model2.encoder.encoder_sequtial.layers[0].kernel

<tf.Variable 'conv2d_6/kernel:0' shape=(3, 3, 4, 64) dtype=float32, numpy=
array([[[[2., 2., 2., ..., 2., 2., 2.],
         [2., 2., 2., ..., 2., 2., 2.],
         [2., 2., 2., ..., 2., 2., 2.],
         [2., 2., 2., ..., 2., 2., 2.]],

        [[2., 2., 2., ..., 2., 2., 2.],
         [2., 2., 2., ..., 2., 2., 2.],
         [2., 2., 2., ..., 2., 2., 2.],
         [2., 2., 2., ..., 2., 2., 2.]],

        [[2., 2., 2., ..., 2., 2., 2.],
         [2., 2., 2., ..., 2., 2., 2.],
         [2., 2., 2., ..., 2., 2., 2.],
         [2., 2., 2., ..., 2., 2., 2.]]],


       [[[2., 2., 2., ..., 2., 2., 2.],
         [2., 2., 2., ..., 2., 2., 2.],
         [2., 2., 2., ..., 2., 2., 2.],
         [2., 2., 2., ..., 2., 2., 2.]],

        [[2., 2., 2., ..., 2., 2., 2.],
         [2., 2., 2., ..., 2., 2., 2.],
         [2., 2., 2., ..., 2., 2., 2.],
         [2., 2., 2., ..., 2., 2., 2.]],

        [[2., 2., 2., ..., 2., 2., 2.],
         [2., 2., 2., ..., 2., 2., 2.],
         [2., 2., 2., ..., 2., 2.

[WARN] [1602434542.600679]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1602435743.401657]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1602435830.842738]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1602435898.394446]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
